# Mushroom Stew

## Develop a mushroom stew, that is visually appealing, pleasingly smelling, and preferably non-toxic.

* Explore the fields, which ones could affect the taste or smell, which can be ignored? 
* Which fields may affect if the stew is visually appealing? 
* Use graphics to support your choices

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
from pandas_profiling import ProfileReport
from urllib.request import urlopen
import json
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

## Load Dataset, Explore and Display Features

In [ ]:
col_names=['class','cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor','gill-attachment',\
           'gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring',\
           'stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type',\
           'veil-color','ring-number','ring-type','spore-print-color','population','habitat'] 

mushroom_df = pd.read_csv('expanded.csv', names=col_names, header=None)

In [ ]:
pd.set_option("display.max_columns", None)
mushroom_df.head(5)

In [ ]:
mushroom_df.dtypes

In [ ]:
mushroom_df.describe()

In [ ]:
profile = ProfileReport(mushroom_df)
profile

## Dataframe Expanded into Dummy Variables

In [ ]:
df_dummy = pd.get_dummies(mushroom_df, columns=col_names)

df_dummy.head()

## Correlation Heatmaps

In [ ]:
cap_xtab = pd.crosstab(mushroom_df['class'],\
                   columns=[mushroom_df['cap-shape'], mushroom_df['cap-surface'], \
                            mushroom_df['cap-color'], mushroom_df['bruises?']]) 

gill_xtab = pd.crosstab(mushroom_df['class'],\
                    columns=[mushroom_df['odor'],mushroom_df['gill-attachment'],\
                            mushroom_df['gill-spacing'], mushroom_df['gill-size'], \
                            mushroom_df['gill-color']])

stalk_xtab = pd.crosstab(mushroom_df['class'],\
                     columns=[mushroom_df['stalk-shape'],mushroom_df['stalk-root'],\
                            mushroom_df['stalk-surface-above-ring'],\
                            mushroom_df['stalk-surface-below-ring'],\
                            mushroom_df['stalk-color-above-ring'],\
                            mushroom_df['stalk-color-below-ring']])

other_xtab = pd.crosstab(mushroom_df['class'],\
                    columns=[mushroom_df['veil-type'],mushroom_df['veil-color'],\
                            mushroom_df['ring-number'],mushroom_df['spore-print-color'],
                            mushroom_df['population'], mushroom_df['habitat']])


In [ ]:
gill_xtab

In [ ]:
sns.heatmap(cap_xtab)

In [ ]:
sns.heatmap(gill_xtab)

In [ ]:
sns.heatmap(stalk_xtab)

In [ ]:
sns.heatmap(other_xtab)

## Feature Engineering

### The field, veil-type, doesn't contribute any information (all are the same value) and can be dropped from the dataset

In [ ]:
mushroom_df_adj = mushroom_df.drop('veil-type', axis=1)
mushroom_df_adj.shape

### Label encode the "class" column and encode the input columns using get_dummies

In [ ]:
# Don't convert "class" column to dummies, convert to Binary instead using LabelEncoder

# label_encoder object
label_encoder =LabelEncoder()
# Encode labels in column. 
mushroom_df_adj['class']= label_encoder.fit_transform(mushroom_df_adj['class']) # 0 is Edible, 1 is Poisonous

col_names_dummies=['cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor','gill-attachment',\
           'gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring',\
           'stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring',\
           'veil-color','ring-number','ring-type','spore-print-color','population','habitat'] 

In [ ]:
# drop_first = True helps avoid multicollinearity
df_dummy2 = pd.get_dummies(mushroom_df_adj, columns=col_names_dummies, drop_first=True)
df_dummy2.head()

## Run 6 Classifier Models

In [ ]:
y = df_dummy2["class"].values
X = df_dummy2.drop(["class"], axis=1).values
# Split data for train and test.
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [ ]:
def classification(method, x_dat, y_dat, **params): # I took classification type out but we might want it)
    
    # Split data for train and test
    X_train, X_test, y_train, y_test = train_test_split(x_dat,y_dat,random_state=42,test_size=0.2)
    
    #fit model
    mod = method(**params)
    mod.fit(X_train, y_train)
    y_pred = mod.predict(X_test)
    
    #print results
    print("Results for {}:".format(str(method)))
    print(classification_report(y_pred, y_test))
    print("Test Accuracy: {}%".format(round(mod.score(X_test,y_test)*100,2)))
    
    #print confusion matrix
    y_pred_rf = y_pred
    y_true_rf = y_test
    cm = confusion_matrix(y_true_rf, y_pred_rf)
    f, ax = plt.subplots(figsize =(5,5))
    sns.heatmap(cm,annot = True,linewidths=0.5,linecolor="red",fmt = ".0f",ax=ax)
    plt.xlabel("y_pred_rf")
    plt.ylabel("y_true_rf")
    plt.title('Confusion Matrix')
    plt.show()
    
    if method == DecisionTreeClassifier:
        print('Feature Importance Plot')
        sns.barplot(y=x_dat.columns, x=mod.feature_importances_)
        plt.xlabel('Mean Decrease Gini')
        plt.show()
        
    if method == DecisionTreeClassifier:
        plot_tree(mod.fit(X_train, y_train))

In [ ]:
x_mushroom = df_dummy2.drop(["class"], axis=1)
y_mushroom = df_dummy2["class"]

In [ ]:
classification(DecisionTreeClassifier, x_mushroom, y_mushroom)

In [ ]:
classification(SVC, x_mushroom, y_mushroom, random_state=42, gamma="auto") # check on parameters, did better with default

In [ ]:
classification(KNeighborsClassifier, x_mushroom, y_mushroom, n_neighbors=5)

In [ ]:
classification(LogisticRegression, x_mushroom, y_mushroom) # on parameters

In [ ]:
classification(GaussianNB, x_mushroom, y_mushroom)

In [ ]:
classification(RandomForestClassifier, x_mushroom, y_mushroom, n_estimators=100, random_state=42) #check on parameters

In [ ]:
classification(LinearDiscriminantAnalysis, x_mushroom, y_mushroom) #check parameters